In [19]:
import pandas as pd
from pathlib import Path
import sys
import numpy as np
from tqdm import tqdm
from re import search
sys.path.extend([str(Path.home() / "PycharmProjects/modeling")])

from analysis.wh.packaging.func import box, box_name, box_dict, find_optimal_box, convert_dim_quantity

In [20]:
# data
path = Path.home() / 'OneDrive - Seagroup/wh/packaging/package.csv'
col = ['order_number', 'main_cate', 'sub_cate', 'mt_sku_id', 'sku_name', 'height_m_sku', 'length_m_sku',
       'width_m_sku', 'actual_quantity_ob', 'packaging_id']
package = pd.read_csv(path, low_memory=False, usecols=col)
package.query(f'packaging_id == {[*box_dict.keys()]}', inplace=True)

for i in ['width_m_sku', 'height_m_sku', 'length_m_sku']:
    package.eval(f'{i} = {i} * 100', inplace=True)
package['main_cate'].fillna('None', inplace=True)
package.head()

,order_number,mt_sku_id,sku_name,main_cate,sub_cate,height_m_sku,length_m_sku,width_m_sku,actual_quantity_ob,packaging_id
0,OBVNW000211104008888,4695157170_26869585151,"Sữa dưỡng thể giúp săn da, dưỡng trắng Nivea b...",Health & Beauty,Bath & Body care,5.0,12.0,7.0,1,S3
1,OBVNW000211014008712,8180632018_101095863639,[Mã FMCGMALL giảm 8% đơn từ 250K] Nước tẩy tra...,Health & Beauty,Skincare,18.0,5.5,5.5,2,S3
2,OBVNW000211213030062,4695157170_26869585151,"Sữa dưỡng thể giúp săn da, dưỡng trắng Nivea b...",Health & Beauty,Bath & Body care,5.0,12.0,7.0,1,S3
3,OBVNN000211011020964,9180620496_111095575303,[HB GIFT] Nước tẩy trang dưỡng trắng Senka All...,Health & Beauty,Skincare,3.0,12.0,5.0,1,S3
4,OBVNW000211102003210,4695157170_26869585151,"Sữa dưỡng thể giúp săn da, dưỡng trắng Nivea b...",Health & Beauty,Bath & Body care,5.0,12.0,7.0,1,S3


In [21]:
# buffer
mask = package['sub_cate'] == "Laundry"

buffer = {}
for o, s in tqdm(package.loc[mask, ['order_number', 'sku_name']].values, desc='buffer'):
    if search('túi', s.lower()):
        buffer.update({o: s})

for i in ['height_m_sku', 'length_m_sku', 'width_m_sku']:
    package[i] = [dim * 0.42 if o in buffer else dim for dim, o in package[[i, 'order_number']].values]

buffer: 100%|██████████| 65832/65832 [00:00<00:00, 221514.89it/s]


In [22]:
# multi order
muilti_item = package.order_number.value_counts().to_frame().query('order_number > 1').index.tolist()
df_multi = package.query(f'order_number == {muilti_item}').copy()
df_multi.sort_values(by=['order_number'], inplace=True)

# single
df_single = package.query(f'order_number != {muilti_item} & actual_quantity_ob == 1').copy()
df_single.sort_values(by=['order_number'], inplace=True)

print('multi', df_multi.shape, len(df_multi.order_number.unique()))
print('single', df_single.shape, len(df_single.order_number.unique()))

multi (823318, 10) 332241
single (605654, 10) 605654


In [23]:
# algorithm
col = ['order_number', 'height_m_sku', 'length_m_sku', 'width_m_sku']
order_single = {o: [h, l, w] for o, h, l, w in tqdm(df_single[col].values)}
quant_single = {o: q for o, q in tqdm(df_single[['order_number', 'actual_quantity_ob']].values)}
order_single_convert = {i: convert_dim_quantity(order_single[i], quant_single[i]) for i in tqdm(order_single)}
order_single_box = {o: find_optimal_box(v, box_name, box, box_dict) for o, v in tqdm(order_single_convert.items())}

100%|██████████| 605654/605654 [02:00<00:00, 5037.46it/s]


In [24]:
# final result
final_single = pd.DataFrame.from_dict(order_single_box, orient='index', columns=['box_cost', 'box_vol']).reset_index().rename(columns={'index': 'order_number'})

# dim
final_single['order_dim'] = final_single['order_number'].map(order_single_convert)
old_box = {i: v for i, v in package[['order_number', 'packaging_id']].values}
final_single['box_base'] = final_single['order_number'].map(old_box)

for i in ['cost', 'vol', 'base']:
    final_single[f'price_box_{i}'] = final_single[f'box_{i}'].map(lambda x: box_dict[x][3] if x and x in box_dict else 0)
    final_single[f'vol_box_{i}'] = final_single[f'box_{i}'].map(lambda x: box_dict[x][4] if x and x in box_dict else 0)
final_single[f'box_base_dim'] = final_single[f'box_base'].map(lambda x: np.sort(box_dict[x][:3]).tolist() if x and x in box_dict else 0)

# check if fit
final_single['is_fit'] = [np.all(b >= o) for o, b in final_single[['order_dim', 'box_base_dim']].values]

# num item
final_single[f'num_item'] = 1

# merge
final_single = final_single.merge(df_single[['order_number', 'mt_sku_id', 'sku_name', 'main_cate', 'sub_cate']], on='order_number', how='left')

In [25]:
tmp = final_single.query('is_fit == False & sub_cate == "Laundry"').copy()
lst = [o - b for o, b in tmp[['order_dim', 'box_base_dim']].values]
dif = pd.DataFrame(lst)
dif.describe()

,0,1,2
count,697.000000,697.000000,697.000000
mean,-9.577590,4.366011,-2.886083
std,5.040802,4.556723,7.269580
min,-16.900000,-12.000000,-20.000000
25%,-12.900000,1.000000,-5.000000
50%,-10.000000,3.000000,-3.000000
75%,-4.540000,9.000000,2.000000
max,1.000000,11.000000,10.000000


In [26]:
final_single.is_fit.value_counts(normalize=True)

True     0.909285
False    0.090715
Name: is_fit, dtype: float64

In [27]:
final_single.query('is_fit == False').sub_cate.value_counts(normalize=True)

Skincare                                0.412826
Hair care                               0.144921
Bath & Body care                        0.058816
Kitchen & Dinning                       0.049153
Milk 24 months and ups                  0.042505
                                          ...   
Supplement and Baby foods 0-6 months    0.000018
Glasses                                 0.000018
Shoes Accessories                       0.000018
Batteries, Charger & Cables             0.000018
Sport Clothes                           0.000018
Name: sub_cate, Length: 86, dtype: float64

In [28]:
final_single.query('order_number == "OBVNW000211228015311"')

,order_number,box_cost,box_vol,order_dim,box_base,price_box_cost,vol_box_cost,price_box_vol,vol_box_vol,price_box_base,vol_box_base,box_base_dim,is_fit,num_item,mt_sku_id,sku_name,main_cate,sub_cate
605649,OBVNW000211228015311,SA2,SA2,"[4.0, 16.0, 20.0]",S3,0.059,2550,0.059,2550,0.0687,3750,"[10, 15, 25]",False,1,10719911127_83610356338,Bộ đôi gội xả Phục hồi ngăn rụng tóc Premium R...,Health & Beauty,Hair care


In [29]:
csv = final_single.query('is_fit == False').copy()
for i, v in enumerate(['dim_min', 'dim_med', 'dim_max']):
    csv[v] = csv['order_dim'].str[i]

for i, v in enumerate(['box_dim_min', 'box_dim_med', 'box_dim_max']):
    csv[v] = csv['box_base_dim'].str[i]
csv.drop(columns=['order_dim', 'box_base_dim'], inplace=True)

In [30]:
csv.to_csv(path.parent / 'not_fit_single.csv', index=False, encoding='utf-8-sig')

In [31]:
csv

,order_number,box_cost,box_vol,box_base,price_box_cost,vol_box_cost,price_box_vol,vol_box_vol,price_box_base,vol_box_base,...,mt_sku_id,sku_name,main_cate,sub_cate,dim_min,dim_med,dim_max,box_dim_min,box_dim_med,box_dim_max
36,OBVNN000210925001710,SA2,SA2,S3,0.0590,2550,0.0590,2550,0.0687,3750,...,8680620533_66535143642,[HB Gift] Ví da cầm tay cao cấp Senka:,Men Clothes,Bags,0.3,16.0,24.0,10,15,25
63,OBVNN000210925001999,"5L, 1L",S4,S7,0.1327,21587,0.1892,15000,0.1263,7800,...,10819911178_111095400409,[COMBO TIẾT KIỆM] Bộ sản phẩm chăm sóc ngăn rụ...,Health & Beauty,Hair care,10.0,19.5,28.0,15,20,26
66,OBVNN000210925002009,SA2,SA2,S2,0.0590,2550,0.0590,2550,0.0378,1600,...,6695144305_36535142611,[Mua 1 tặng 1] Tinh chất chống nắng Senka Perf...,Health & Beauty,Skincare,3.0,15.0,20.0,8,10,20
69,OBVNN000210925002032,SA2,SA2,S2,0.0590,2550,0.0590,2550,0.0378,1600,...,11719914034_56535100981,Combo 3 Bàn chải đánh răng P/S Detox Trắng Răn...,Health & Beauty,Oral care,4.0,7.0,21.0,8,10,20
94,OBVNN000210925002306,"5L, 1L",S4,S7,0.1327,21587,0.1892,15000,0.1263,7800,...,10819911178_111095400409,[COMBO TIẾT KIỆM] Bộ sản phẩm chăm sóc ngăn rụ...,Health & Beauty,Hair care,10.0,19.5,28.0,15,20,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605618,OBVNW000211228014106,SA2,SA2,S3,0.0590,2550,0.0590,2550,0.0687,3750,...,10719911127_83610356338,Bộ đôi gội xả Phục hồi ngăn rụng tóc Premium R...,Health & Beauty,Hair care,4.0,16.0,20.0,10,15,25
605624,OBVNW000211228014238,SA2,SA2,S3,0.0590,2550,0.0590,2550,0.0687,3750,...,10719911127_83610356338,Bộ đôi gội xả Phục hồi ngăn rụng tóc Premium R...,Health & Beauty,Hair care,4.0,16.0,20.0,10,15,25
605625,OBVNW000211228014287,SA2,SA2,S3,0.0590,2550,0.0590,2550,0.0687,3750,...,10719911127_83610356338,Bộ đôi gội xả Phục hồi ngăn rụng tóc Premium R...,Health & Beauty,Hair care,4.0,16.0,20.0,10,15,25
605649,OBVNW000211228015311,SA2,SA2,S3,0.0590,2550,0.0590,2550,0.0687,3750,...,10719911127_83610356338,Bộ đôi gội xả Phục hồi ngăn rụng tóc Premium R...,Health & Beauty,Hair care,4.0,16.0,20.0,10,15,25


In [32]:
final_single.query('is_fit == False & sub_cate == "Laundry"')

,order_number,box_cost,box_vol,order_dim,box_base,price_box_cost,vol_box_cost,price_box_vol,vol_box_vol,price_box_base,vol_box_base,box_base_dim,is_fit,num_item,mt_sku_id,sku_name,main_cate,sub_cate
2425,OBVNN000210927000415,SA2,SA2,"[5.04, 10.92, 13.86]",S2,0.0590,2550,0.0590,2550,0.0378,1600,"[8, 10, 20]",False,1,9880616836_46534907755,Ariel Matic nước giặt Túi 3.5KG/3.2KG:Đậm Đặc ...,Home care,Laundry
2792,OBVNN000210927001987,SA2,SA2,"[4.62, 10.5, 14.28]",S2,0.0590,2550,0.0590,2550,0.0378,1600,"[8, 10, 20]",False,1,3995203353_101095410068,Nước xả vải Downy Hương Nước Hoa túi 3.5L (mới...,Home care,Laundry
2850,OBVNN000210927002253,"5L, 1L",S5,"[3.1, 26.0, 32.0]",S3,0.1327,21587,0.2134,15000,0.0687,3750,"[10, 15, 25]",False,1,9880598865_46534908449,Nước giặt Ariel Cửa Trước 3.6 KG/3.25 KG/3.05K...,Home care,Laundry
3440,OBVNN000210928000287,TD-5L,TD-5L,"[7.000000000000001, 17.0, 32.5]",S4,0.1327,13005,0.1327,13005,0.1892,15000,"[20, 25, 30]",False,1,9580620324_83610687147,Nước giặt gốc thực vật Seventh Generation 2L (...,Home care,Laundry
7617,OBVNN000211001004999,"5L, 1L",S5,"[3.1, 26.0, 32.0]",S7,0.1327,21587,0.2134,15000,0.1263,7800,"[15, 20, 26]",False,1,9880598865_46534908449,Nước giặt Ariel Cửa Trước 3.6 KG/3.25 KG/3.05K...,Home care,Laundry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599394,OBVNW000211227002210,"5L, 1L",S5,"[7.000000000000001, 18.5, 32.0]",TD-5L,0.1327,21587,0.2134,15000,0.1327,13005,"[17, 17, 45]",False,1,9580620324_83610687146,Nước giặt gốc thực vật Seventh Generation 2L (...,Home care,Laundry
600165,OBVNW000211227004738,"5L, 1L",S4,"[9.0, 21.0, 30.0]",TD-5L,0.1327,21587,0.1892,15000,0.1327,13005,"[17, 17, 45]",False,1,9580620284_56535109181,Nước Giặt Surf Trắng Sạch Ngát Hương Chai 3.6 ...,Home care,Laundry
602012,OBVNW000211227011198,"5L, 1L",S5,"[10.0, 20.0, 32.0]",TD-5L,0.1327,21587,0.2134,15000,0.1327,13005,"[17, 17, 45]",False,1,3995204218_66535110248,Nước Xả Vải Em Bé Comfort Cho Da Nhạy Cảm Với ...,Home care,Laundry
602715,OBVNW000211227014195,"5L, 1L",S5,"[7.000000000000001, 18.5, 32.0]",TD-4L,0.1327,21587,0.2134,15000,0.1327,8960,"[16, 16, 35]",False,1,9580620324_83610687146,Nước giặt gốc thực vật Seventh Generation 2L (...,Home care,Laundry


In [34]:
# report
from core_pro.ultilities import update_df

report = pd.DataFrame()
for i in ['cost', 'vol', 'base']:
    dict_ = {'order_number': 'nunique', f'price_box_{i}': 'sum', f'vol_box_{i}': 'sum'}
    rpt = final_single.groupby(f'box_{i}').agg(dict_).reset_index()
    rpt.columns = ['box_name', 'total_order', 'total_price', 'total_vol']
    rpt['model'] = i
    report = report.append(rpt)

# export
sh = '1RXVHMSA9oaVWMWXCDfa5GcZMbP9USvkwMPMRwzizb8E'
update_df(report, 'raw_single', sh)

Successfully clean sheet: 1RXVHMSA9oaVWMWXCDfa5GcZMbP9USvkwMPMRwzizb8E at raw_single!A1:E
Status: Sheet updating..
Updated Google Sheet value at: raw_single!A1:E


In [35]:
report

,box_name,total_order,total_price,total_vol,model
0,"5L, 1L",19470,2583.6690,420298890,cost
1,S0,141471,4272.4242,72150210,cost
2,S10,7250,3677.9250,533600000,cost
3,S2,227911,8615.0358,364657600,cost
4,S3,46739,3210.9693,175271250,cost
5,S5,1758,375.1572,26370000,cost
6,S6,4113,1060.3314,123390000,cost
7,S7,45702,5772.1626,356475600,cost
8,S9,257,90.7210,6939000,cost
9,SA2,90144,5318.4960,229867200,cost
